In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",          # Phi-3 2x faster!d
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

# 모델 및 토크나이저 로드
model_name = "unsloth/Meta-Llama-3.1-8B"  # 사용할 모델 이름
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2024-08-17 14:28:16.264733: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-17 14:28:16.264779: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-17 14:28:16.265737: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-17 14:28:16.270641: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-17 14:28:17.058211: W tensorflow/compiler/tf2

==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.0.
   \\   /|    GPU: NVIDIA RTX A5000. Max memory: 23.677 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 101,
    use_rslora = False,
    loftq_config = None,
)


Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [23]:
# Alpaca 포맷 정의
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# EOS 토큰 설정
EOS_TOKEN = tokenizer.eos_token

# 데이터 포맷팅 함수 정의
def formatting_prompts_func(examples):
    questions = examples["Question"]
    answers = examples["Answer"]
    texts = []
    for question, answer in zip(questions, answers):
        text = alpaca_prompt.format("Answer the question", question, answer) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}
from datasets import load_dataset
# CSV 데이터셋 로드
dataset = load_dataset('csv', data_files="/home/jovyan/temp/open/train.csv", split='train')

# 포맷팅 함수 적용
dataset = dataset.map(formatting_prompts_func, batched=True)

# 포맷팅된 데이터 확인
print(dataset[14])


{'SAMPLE_ID': 'TRAIN_014', 'Source': '1-1 2024 주요 재정통계 1권', 'Source_path': './train_source/1-1 2024 주요 재정통계 1권.pdf', 'Question': '보전지출이란 무엇을 의미하며, 어떤 상황에서 발생하는가?', 'Answer': '보전지출은 회계 또는 기금의 민간차입 상환(국채상환), 남은 자금의 금융기관 예치(기금여유자금 운용) 등을 의미하며, 민간차입 상환이나 자금의 운용 상황에서 발생한다.', 'text': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nAnswer the question\n\n### Input:\n보전지출이란 무엇을 의미하며, 어떤 상황에서 발생하는가?\n\n### Response:\n보전지출은 회계 또는 기금의 민간차입 상환(국채상환), 남은 자금의 금융기관 예치(기금여유자금 운용) 등을 의미하며, 민간차입 상환이나 자금의 운용 상황에서 발생한다.<|end_of_text|>'}


In [5]:
# 모델 훈련 설정
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,  # 짧은 시퀀스에 대해 훈련을 빠르게 할 수 있습니다.
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,  # 최대 훈련 스텝 수
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=101,
        output_dir="outputs",
    ),
)


max_steps is given, it will override any value given in num_train_epochs


In [6]:
# 훈련 실행
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 496 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.919800
2,2.473700
3,2.658600
4,2.561000
5,2.502700
6,2.215600
7,2.145200
8,1.810800
9,1.608000
10,1.721100


In [43]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Answer the question", # instruction
        "asdf", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n보건지출?\n\n### Input:\n\n\n### Response:\n보건지출은 국민보건에 대한 재정지원을 위해 보건복지부와 국립의료원, 보건복지부 산하 중앙보건위생본부, 보건복지부 산하 중앙보건위생본부 산하 중앙보건위생본부 산하']

In [ ]:
#보전지출은 회계 또는 기금의 민간차입 상환(국채상환), 남은 자금의 금융기관 예치(기금여유자금 운용) 등을 의미하며, 민간차입 상환이나 자금의 운용 상황에서 발생한다

In [42]:
model.save_pretrained("llama3_model_파인튜닝") # Local saving
tokenizer.save_pretrained("llama3_model_파인튜닝")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('llama3_model_파인튜닝/tokenizer_config.json',
 'llama3_model_파인튜닝/special_tokens_map.json',
 'llama3_model_파인튜닝/tokenizer.json')